In [2]:
%%capture
from scripts import *
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import re
from selenium import webdriver
from bs4 import BeautifulSoup

# ==================== GLOBAL VARIABLES
# Set Chrome options:
chrome_options = Options()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument("--incognito")
# chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-application-cache")
chrome_options.add_argument("--force-device-scale-factor=0.75")  # Set zoom level to 125%
# chrome_options.add_argument("--high-dpi-support=1")  # Ensure DPI support
chrome_options.add_argument("--disable-infobars")  # Disable info bars
chrome_options.add_argument("--disable-extensions")  # Disable extensions
# chrome_options.add_argument("--disable-cache")
chrome_options.add_argument("--disk-cache-size=0")

In [4]:
df = pd.read_csv("vietnam_hotel.csv")

from tqdm import tqdm

# Initialize a progress bar
progress_bar = tqdm(total=len(df), desc="Processing Hotels", unit="hotel")
processed_hotel_ids = set()

if os.path.exists("reviews"):
    processed_hotel_ids = {
        os.path.splitext(filename)[0]  # Extract hotel_id from filenames
        for filename in os.listdir("reviews")
        if filename.endswith(".csv")  # Ensure it's a CSV file
    }

for index, row in df.iterrows():
    hotel_id = str(row['hotel_id'])
    # Skip processing if the hotel_id is already in the list
    if hotel_id in processed_hotel_ids:
        continue
        
    url = row['url_edited']

    driver = webdriver.Chrome(chrome_options)
    driver.get(url)
    driver.get(re.sub(r"/vi-vn/", "/", driver.current_url))
    
    ###### Extract informations and reviews summary:
    time.sleep(random.uniform(3, 7))
    # Ensure "errors" directory exists
    if not os.path.exists("errors"):
        os.makedirs("errors")
    
    # Open the error file for appending
    with open("errors/crawl_again_reviews.csv", mode="a", newline="", encoding="utf-8-sig") as error_file:
        error_writer = csv.writer(error_file)
        if not file_exists:
            error_writer.writerow(["hotel_id"])  # Write header only once
            
        try:
            # Background remove:
            searchbox_element = driver.find_element(By.CSS_SELECTOR, '.Searchbox.Searchbox--horizontal')
            actions = ActionChains(driver)
            actions.move_to_element(searchbox_element).click().perform()
            time.sleep(2)
            actions.move_to_element(searchbox_element).click().perform()
            time.sleep(random.uniform(2, 4))
        
            # Extract data from webpage:
            page_source = driver.page_source
            information = get_information(page_source)
            time.sleep(random.uniform(1, 3))
            review_summary = get_review_summary(page_source)
            merge_df = {**information, **review_summary}
            
            # Save to dataframe:
            csv_file = "hotel_information.csv"
            # Check if the file exists
            file_exists = os.path.exists(csv_file)
            with open(csv_file, mode='a', newline='', encoding='utf-8-sig') as file:
                writer = csv.writer(file)
            
                # Write the header if the file doesn't exist
                if not file_exists:  # File doesn't exist, write the header
                    header = ['hotel_id'] + list(merge_df.keys())
                    writer.writerow(header)
            
                # Write the row with hotel_id and merged data
                row = [hotel_id] + list(merge_df.values())
                writer.writerow(row)
        except:
            pass
        # Looping reviews:
        try:
            try:
                driver.find_elements(By.CSS_SELECTOR, "li.NavBar__menu[data-element-name='customer-reviews-panel-navbar-menu']")[1].click()
            except:
                try:
                    driver.find_elements(By.XPATH, '//*[@id="customer-reviews-panel-tab-5"]')[1].click()
                except:
                    pass
            time.sleep(2)
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'button[data-element-name="review-paginator-next"]')
            except:
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'i.ficon.ficon-24.ficon-carrouselarrow-right')
                except:
                    pass
                    
            driver.find_element(By.ID, 'review-sort-id').click()
            actions.send_keys(Keys.UP).perform()
            time.sleep(random.uniform(1, 3))
    
            actions.send_keys(Keys.UP).perform()
            time.sleep(0.5)
            actions.send_keys(Keys.UP).perform()
            time.sleep(0.5)
            actions.send_keys(Keys.ENTER).perform() 
            
            if not os.path.exists("errors"):
                os.makedirs("errors")
            if not os.path.exists("reviews"):
                os.makedirs("reviews")
                
            while True:
                try:
                    # Click the next button
                    next_button.click()
                    time.sleep(random.uniform(1, 3))
                    # Get the page source
                    page_source = driver.page_source
                    time.sleep(random.uniform(1, 3))
                    # Extract reviews
                    reviews = get_reviews(page_source)
                    
                    # Save reviews to CSV file (appending to {hotel_id}.csv)
                    review_file_path = f"reviews/{hotel_id}.csv"
                    with open(review_file_path, mode="a", newline="", encoding="utf-8-sig") as review_file:
                        review_writer = csv.DictWriter(review_file, fieldnames=reviews[0].keys())
                        if review_file.tell() == 0:
                            review_writer.writeheader()  # Write header only if the file is empty
                        review_writer.writerows(reviews)
                except:
                    driver.close()
                    break      
        except Exception as e:
            error_writer.writerow([hotel_id])
    progress_bar.update(1)
    driver.close()
    
progress_bar.close()


Processing Hotels:   0%|                                                      | 3/9406 [07:40<400:53:02, 153.48s/hotel]

Processing Hotels:   0%|                                                        | 2/9406 [00:32<44:30:06, 17.04s/hotel]

KeyboardInterrupt: 